### Imports

In [1]:
import os, shutil
import csv, json
import ogr, osr
from tqdm import tqdm_notebook
from collections import defaultdict, Counter
from matplotlib import pyplot as plt, figure
import copy
import numpy as np
from operator import itemgetter

import overpy

import xml.etree.ElementTree as ET

ogr.UseExceptions()
osr.UseExceptions()

### Paths

In [2]:
# Parent directory
cwd = os.getcwd()

# zipfile
zipfile = os.path.join(cwd, 'gtfs.zip')

# Child directories
boundaries_dir = os.path.join(cwd, 'boundaries')
gtfs_dir = os.path.join(cwd, 'gtfs')
output_dir = os.path.join(cwd, 'output')

# Create output directory
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir)

# Create gtfs directory
if os.path.exists(gtfs_dir):
    shutil.rmtree(gtfs_dir)
os.makedirs(gtfs_dir)

# Unzip GTFS
shutil.unpack_archive(zipfile, gtfs_dir)

###  Functions

In [3]:
def dict_to_geojson(data, out_path, geom_field, fields_key=None, epsg_id=None):
    
    # Create path
    if os.path.exists(out_path):
        os.remove(out_path)
        
    # Get GeoJSON driver
    driver = ogr.GetDriverByName('GeoJSON')
    
    ds = driver.CreateDataSource(out_path)
    
    spatial_ref = osr.SpatialReference()
    if epsg_id:
        spatial_ref.ImportFromEPSG(4326)
    else:
        spatial_ref.ImportFromEPSG(epsg_id)
        
    # Get geom type
    geom_type = data[0][geom_field].GetGeometryType()
    
    layer = ds.CreateLayer(out_path, geom_type=geom_type, srs=spatial_ref)
    
    # Create the fields
    if fields_key:
        field_names = data[0][fields_key].keys()
        for i, field_name in enumerate(field_names): 
            layer.CreateField(ogr.FieldDefn(field_name, ogr.OFTString))
    
    layer_defn = layer.GetLayerDefn()
    
    for item in data:
        feature = ogr.Feature(layer_defn)
        
        if fields_key:
            for field_name in field_names:
                feature.SetField(field_name, item[fields_key][field_name])
        
        feature.SetGeometry(item[geom_field])
        
        layer.CreateFeature(feature)
        
def objects_to_xml(path, bounds=None, nodes=None, ways=None, relations=None):
    root = ET.Element("osm")
    pass

def reproject_geometry(geom, in_epsg, out_epsg, return_wkt=False):
    import ogr, osr

    source = osr.SpatialReference()
    source.ImportFromEPSG(in_epsg)

    target = osr.SpatialReference()
    target.ImportFromEPSG(out_epsg)

    transform = osr.CoordinateTransformation(source, target)

    geom.Transform(transform)

    if return_wkt:
        return geom.ExportToWkt()
    else:
        return geom
    
def write_geometry_to_geojson(geom, out_path):
    if os.path.exists(out_path):
        os.remove(out_path)
    
    driver = ogr.GetDriverByName('GeoJSON')
    ds = driver.CreateDataSource(out_path)
    
    geom_type = geom.GetGeometryType()
    
    layer = ds.CreateLayer(out_path, geom_type=geom_type)
    layer_defn = layer.GetLayerDefn()
    
    feature = ogr.Feature(layer_defn)
    feature.SetGeometry(geom)
    layer.CreateFeature(feature)

### Overpy Query Templates

*Overpass doesn't allow Geocoding like Turbo Overpass does so I've gathered the city relation ids beforehand to use directly with overpass*

In [4]:
api = overpy.Overpass()

relation_to_area_factor = 3600000000

region_ids = {
    "Laval": 3532125 + relation_to_area_factor,
    "Montreal": 1571328 + relation_to_area_factor
}

bus_stop_tmpl = """
    area({})->.searchArea;
    (
    node["highway"="bus_stop"](area.searchArea);
    way["highway"="platform"](area.searchArea);

    node["public_transport"="platform"]["bus"="yes"](area.searchArea);
    node["public_transport"="stop_position"]["bus"="yes"](area.searchArea);
    
    way["amenity"="shelter"](area.searchArea);
    node["amenity"="shelter"](area.searchArea);
    );
    out body;
"""

service_route_tmpl = """
    area({})->.searchArea;
    (
    relation["type"="route"]["route"="bus"](area.searchArea);
    );
    out body;
"""

master_route_tmpl = """
    area({})->.searchArea;
    (
    relation["type"="master_route"]["route_master"="bus"](area.searchArea);
    );
    out body;
"""

### Load Boundaries into memory

In [5]:
boundaries = {}

boundary_files = os.listdir(boundaries_dir)

for boundary_file in tqdm_notebook(boundary_files):
    path = os.path.join(boundaries_dir, boundary_file)
    city = boundary_file[:-4]
    
    with open(path) as f:
        geom = ogr.CreateGeometryFromWkt(f.read())
        boundaries[city] = geom

HBox(children=(IntProgress(value=0, max=2), HTML(value='')))

### Get existing data from OSM using OverPy

#### Get existing stops

In [6]:
existing_stops = []

stops_result_laval = api.query(bus_stop_tmpl.format(region_ids['Laval']))
stops_result_montreal = api.query(bus_stop_tmpl.format(region_ids['Montreal']))

In [7]:
for node in stops_result_laval.nodes:
    existing_stop = {
        'id': node.id,
        'lat': node.lat,
        'lon': node.lon,
        'tags': node.tags,
        'city': 'Laval'
    }
    existing_stops.append(existing_stop)
    
for node in stops_result_montreal.nodes:
    existing_stop = {
        'id': node.id,
        'lat': node.lat,
        'lon': node.lon,
        'tags': node.tags,
        'city': 'Montreal'
    }
    existing_stops.append(existing_stop)

#### Get existing routes

In [50]:
existing_routes = []
existing_route_masters = []

routes_result_laval = api.query(service_route_tmpl.format(region_ids['Laval']))
route_master_result_laval = api.query(master_route_tmpl.format(region_ids['Laval']))

In [51]:
routes_result_laval.relations[0].members[0]._type_value

'way'

In [60]:
for relation in route_master_result_laval.relations:
#     if hasattr(relation.tags, 'operator'):
#         print(relation.tags.operator)
#     if hasattr(relation.tags, 'network'):
#         print(relation.tags.network)
        
    print(relation)

### Load GTFS text files to memory

In [12]:
gtfs_data = {}

filenames = os.listdir(gtfs_dir)

for filename in tqdm_notebook(filenames):
    table_name = filename[:-4]
    path = os.path.join(gtfs_dir, filename)
    gtfs_data[table_name] = {
        "path": path,
    }

    with open(path, encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        
        field_names = next(reader)
        gtfs_data[table_name]["field_names"] = field_names
        
        dict_reader = csv.DictReader(csvfile, fieldnames=field_names)
        data = [row for row in dict_reader]
        gtfs_data[table_name]["data"] = data

HBox(children=(IntProgress(value=0, max=8), HTML(value='')))

#### Print the dictionary information for reference

In [13]:
print('# table: # field names')
for key, value in gtfs_data.items():
    print('{}: {}\n'.format(key, value['field_names']))

# table: # field names
routes: ['route_id', 'agency_id', 'route_short_name', 'route_long_name', 'route_type', 'route_url', 'route_headsign', 'route_color', 'route_text_color']

stop_times: ['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence', 'pickup_type', 'drop_off_type']

stops: ['stop_id', 'stop_code', 'stop_name', 'stop_lon', 'stop_lat', 'location_type', 'stop_display', 'stop_abribus']

shapes: ['shape_id', 'shape_pt_lat', 'shape_pt_lon', 'shape_pt_sequence']

agency: ['agency_id', 'agency_name', 'agency_url', 'agency_timezone', 'agency_lang']

trips: ['route_id', 'service_id', 'trip_id', 'block_id', 'shape_id', 'trip_headsign']

calendar: ['service_id', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday', 'start_date', 'end_date']

calendar_dates: ['service_id', 'date', 'exception_type']



### Deduplicate the stops file
1. Split the stop code from the name
2. Check for name uniqueness
3. Check for location uniqueness (with proximity tolerance - optional)

#### Visualize the stop **names** and **codes** frequency distribution

In [14]:
# stops = gtfs_data['stops']['data']

# unique_stops = []
# stop_name_counter = Counter()
# stop_code_counter = Counter()

# for stop in tqdm_notebook(stops):
#     real_name = stop['stop_name'].split('[')[0].strip()
#     stop_code = stop['stop_code']
#     stop_name_counter[real_name] += 1
#     stop_code_counter[stop_code] += 1
    
# names, name_counts = zip(*stop_name_counter.items())
# codes, code_counts = zip(*stop_code_counter.items())

# bar_height = 10

# plot_specs = {
#     'names': {
#         'positions': np.arange(len(names)),
#         'bar_spacing': 2 * np.arange(len(names)) * bar_height,
#         'labels': names,
#         'values': name_counts,
#         'title': 'Distribution of stop names'
#     },
#     'codes': {
#         'positions': np.arange(len(codes)),
#         'bar_spacing': 2 * np.arange(len(codes)) * bar_height,
#         'labels': codes,
#         'values': code_counts,
#         'title': 'Distribution of stop codes'
#     }
# }

In [15]:
# # Can't get this to work....moving on

# # plt.figure(figsize=(50,30))
# plt.rcParams.update({'font.size': 10})

# for i, (plot_name, spec) in enumerate(plot_specs.items(), 1):
#     plt.subplot(1, 2, i)
#     plt.barh(spec['positions'], spec['values'], height=bar_height)
#     plt.yticks(spec['bar_spacing'], spec['labels'])
#     plt.title(spec['title'])
#     plt.autoscale()
# plt.tight_layout()
# plt.show()

#### Create unique stops list

In [16]:
stops = gtfs_data['stops']['data']

# existing_count_laval = 0
# existing_count_montreal = 0
existing_count_total = 0

# Extent
multi_point = ogr.Geometry(ogr.wkbMultiPoint)

# Dictionary to group stops by unique location
unique_locations = defaultdict(list)

# List of stops rearranged with uniqueness
unique_stops = []

# Aggregate unique locations
for stop in stops:
    unique_locations[(stop['stop_lon'], stop['stop_lat'])].append(stop)

# Aggregate attributes using unique locations into new stop objects
for i, (unique_location, stops) in enumerate(tqdm_notebook(unique_locations.items()), 1):

    ids = list(set([stop['stop_id'] for stop in stops]))
    codes = list(set([stop['stop_code'] for stop in stops]))

    lon = float(stops[0]['stop_lon'])
    lat = float(stops[0]['stop_lat'])

    # Create point to add to MultiPoint (for JOSM extent later...)
    point = ogr.Geometry(ogr.wkbPoint)
    point.AddPoint(lon, lat)
    multi_point.AddGeometry(point)

    # Create the unique stop according to JOSM format
    action = None

    # While checking for:
    # 1. What city it is ine
    # 2. If a node with the same geometry already exists in OSM
    # 2.1 If node is in Laval, replace the tags
    # 2.2 If node is in Montreal, append, code and name tags

#     # Get the city of the GTFS stop
#     city = None
#     for city_name, city_geom in boundaries.items():
#         if point.Intersects(city_geom):
#             city = city_name

    osm_id = None
    osm_lat = None
    osm_lon = None

    # Determine if node already exists
    for existing_stop in existing_stops:
        existing_point = ogr.Geometry(ogr.wkbPoint)
        existing_point.AddPoint(
            float(existing_stop['lon']), float(existing_stop['lat']))

        if point.Equals(existing_point):
            #             print('Equal point found')
            existing_count_total += 1

            osm_id = existing_stop['id']
            osm_lat = existing_stop['lat']
            osm_lon = existing_stop['lon']
            action = 'modify'

#             print(osm_id)
        else:
            osm_id = str(i * -1)
            osm_lat = str(lat)
            osm_lon = str(lon)

    unique_stop = {
        # props
        'props': {
            'id': osm_id,
            'lon': osm_lon,
            'lat': osm_lat,
        },
        # tags
        'tags': {
            'bus': 'yes',
            'highway': 'bus_stop',
            'name': stops[0]['stop_name'].split('[')[0].strip(),
            'public_transport': 'platform',
            'ref': ';'.join(codes),
            'shelter': 'yes' if stops[0]['stop_abribus'] == '1' else 'no',
        },
        "geom": point,
        # GTFS fields
        "gtfs_props": {
            'stop_id': ','.join(ids),
            'stop_code': ','.join(codes),
            'stop_name': stops[0]['stop_name'].split('[')[0].strip(),
            'stop_lon': lon,
            'stop_lat': lat,
            'location_type': stops[0]['location_type'],
            'stop_display': stops[0]['stop_display'],
            'stop_abribus': stops[0]['stop_abribus'],
        }
    }
    if action:
        unique_stop['props']['action'] = action
        
    unique_stops.append(unique_stop)

HBox(children=(IntProgress(value=0, max=2773), HTML(value='')))

#### Merge stops by proximity (tolerance)

In [ ]:
buffer_distance = 5  # meter
multi_point_utm = reproject_geometry(multi_point.Clone(), 4326, 32618)
buffered_points = multi_point_utm.Buffer(5)
buffered_points_dissolved = buffered_points.UnionCascaded()
buffered_points_dissolved_geo = reproject_geometry(buffered_points_dissolved.Clone(), 32618, 4326)

buffers_file = os.path.join(output_dir, 'buffers.geojson')
write_geometry_to_geojson(buffered_points_dissolved_geo, buffers_file)

stops_file = os.path.join(output_dir, 'gtfs_stops.geojson')
dict_to_geojson(unique_stops, stops_file, 'geom', fields_key='gtfs_props', epsg_id=4326)

In [ ]:
unique_stops_merged = []

for i, buffer in tqdm_notebook(enumerate(buffered_points_dissolved_geo)):
    stops_to_merge = []
    for stop in unique_stops:
        if stop['geom'].Within(buffer):
            stops_to_merge.append(stop)
            
    if len(stops_to_merge) > 1:
        new_stop = stops_to_merge[0].copy()
        merged_codes = []
        merged_ids = []
        
        for stop_to_merge in stops_to_merge:
            merged_codes.extend(stop_to_merge['gtfs_props']['stop_code'].split(','))
            merged_ids.extend(stop_to_merge['gtfs_props']['stop_id'].split(','))
            
            # Copy the first stop in the list and update the keys
            new_stop['gtfs_props']['stop_code'] = ','.join(merged_codes)
            new_stop['gtfs_props']['stop_id'] = ','.join(merged_ids)
            new_stop['tags']['ref'] = ';'.join(merged_codes)
            
        unique_stops_merged.append(new_stop)
        
    if len(stops_to_merge) == 1:
        unique_stops_merged.append(stops_to_merge[0])
            
print('Unique stops before merge: {}'.format(len(unique_stops)))
print('Unique stops after merge: {}'.format(len(unique_stops_merged)))
                

In [ ]:
unique_stops_merged_file = os.path.join(output_dir, 'unique_stops_merged.geojson')
dict_to_geojson(unique_stops_merged, unique_stops_merged_file, 'geom', 'gtfs_props', 4326)

### Consolidate GTFS routes

In [ ]:
routes_data = gtfs_data['routes']['data']
stop_times_data = gtfs_data['stop_times']['data']
trips_data = gtfs_data['trips']['data']

In [ ]:
# Find all trips for a route
route_trips = defaultdict(set)

for trip in trips_data:
    route_trips[trip['route_id']].add(trip['trip_id'])

In [ ]:
# Count the number of stops per trip
stops_counter = Counter()

for stop_time in tqdm_notebook(stop_times_data):
    stops_counter[stop_time['trip_id']] += 1

stops_counter = dict(stops_counter)

In [ ]:
# Find the longest trip per route
longest_route_trips = []

for route_id, trip_ids in tqdm_notebook(route_trips.items()):
    trips = []
    
    for trip_id, stop_count in stops_counter.items():
        if trip_id in trip_ids:
            trips.append((trip_id, stop_count))
            
    trips.sort(key=itemgetter(1), reverse=True)
    
    longest_route_trips.append({
        "route_id": route_id,
        "trip_id": trips[0][0],
        "stops_count": trips[0][1]
    })

In [ ]:
longest_route_trips[0]

In [ ]:
# Add the stop ids to the longest route
for longest_trip in tqdm_notebook(longest_route_trips):
    trip_id = longest_trip['trip_id']
    stops = []
    
    for stop_time in stop_times_data:
        if stop_time['trip_id'] == trip_id:
            stops.append({
                "gtfs_id": stop_time['stop_id'],
                "sequence": int(stop_time['stop_sequence'])
            })
            
    stops.sort(key=itemgetter('sequence'))
    longest_trip['stops'] = stops

In [ ]:
longest_route_trips[0]

In [ ]:
print(len(longest_route_trips))
for longest_route in tqdm_notebook(longest_route_trips):
    print(len(longest_route['stops']))

In [ ]:
# Map the GTFS stops to the merged unique stops
for longest_trip in tqdm_notebook(longest_route_trips):
    for gtfs_stop in longest_trip['stops']:
        for unique_stop in unique_stops_merged:
            unique_stop_ids = unique_stop['gtfs_props']['stop_id'].split(',')
            osm_id = unique_stop['props']['id']
            name = unique_stop['tags']['name']
            if gtfs_stop['gtfs_id'] in unique_stop_ids:
                gtfs_stop.update({
                    "osm_id": osm_id,
                    "name": name
                })

In [ ]:
longest_route_trips[0]['stops']

In [ ]:
for route in longest_route_trips:
    for stop in route['stops']:
        if 'name' not in stop:
            print('no name')
            print(stop)

#### Get the route names and other info

In [ ]:
osm_new_route_id = -10000

for i, longest_route in enumerate(tqdm_notebook(longest_route_trips)):
    route_id = longest_route['route_id']
    route_stops = longest_route['stops']
    
    first_stop = route_stops[0]['name']
    last_stop = route_stops[-1]['name']

    loop_route = 'no'

    if first_stop == last_stop:
        loop_route = 'yes'
        print('loop route found')

    for route in routes_data:
        if route_id == route['route_id']:
            osm_new_route_id -= 1
            longest_route.update({
                "props": {
                    "id": osm_new_route_id
                },
                "tags": {
                    "name": route['route_long_name'],
#                     "ref": route["route_headsign"],
                    "ref": route['route_url'].split('route_id=')[1],
                    "type": "route",
                    "route": "bus",
                    "network": "STL",
                    "operator": "STL",
                    "from": first_stop,
                    "to": last_stop,
                    "round_trip": loop_route,
                    "public_transport:version": 2
                },
                "master_route_ref": route['route_short_name']
            })

In [ ]:
# Sometimes the routes file containes unique ids for trimesters

route_ref_counter = Counter()

for longest_route in longest_route_trips:
    route_ref_counter[longest_route['master_route_ref']] += 1

route_ref_counter

In [ ]:
len(route_ref_counter.keys())

In [ ]:
sum(route_ref_counter.values())

In [ ]:
# need to reduce to 2 routes per ref max
aggregate_longest_routes = defaultdict(list)

for longest_route in longest_route_trips:
    ref = longest_route['master_route_ref']
    aggregate_longest_routes[ref].append(longest_route)

In [ ]:
aggregate_longest_routes.keys()

In [ ]:
# for ref, routes in aggregate_longest_routes.items():
# #     route_ids = [route['route_id'] for route in routes]
# #     print(ref, route_ids)
    
#     a_routes = []
#     j_routes = []
    
#     for route in routes:
#         if route['route_id'].startswith('A'):
#             a_routes.append(route)
#         else:
#             j_routes.append(route)
            
#     for a_route in a_routes:
#         print('a routes:')
#         print(len(a_route['stops']))
#     for j_route in j_routes:
#         print('j routes:')
#         print(len(j_route['stops']))


# Delete August routes
filtered_routes = defaultdict(list)
for ref, routes in aggregate_longest_routes.items():
    if len(routes) == 4:
        for route in routes:
            if route['route_id'].startswith('J'):
                filtered_routes[ref].append(route)
    else:
        for route in routes:
            filtered_routes[ref].append(route)

In [ ]:
for ref, routes in filtered_routes.items():
    print(ref, len(routes))

print(len(filtered_routes.keys()))

In [ ]:
# master_routes = defaultdict(list)

# for route in routes_data:
#     if route['route_id'].startswith('J'):
#         master_routes[route['route_short_name']].append(route)

In [ ]:
# master_routes

In [ ]:
route_relations = []
route_master_relations = []

In [ ]:
osm_new_master_route_id = -100000

for ref, routes in filtered_routes.items():
    osm_new_master_route_id -= 1
    names = set()
    members = []
#     route_ids = []
    for route in routes:
        for gtfs_route in routes_data:
            if route['route_id'] == gtfs_route['route_id']:
                names.add(gtfs_route['route_long_name'].split('Direction')[1].strip())
#                 route_ids.append(route['route_id'])

        member = {
            "props": {
                "type": 'relation',
                "ref": route["props"]["id"],
                "role": ''
            }
        }
        members.append(member)

    names = list(names)

    if len(names) == 2:
        name = '{} - {}'.format(names[0], names[1])
    elif len(names) == 1:
        name = names[0]
        print('Start and End have same name...')
    else:
        print('error')

    route_master_relation = {
        "props": {
            "id": osm_new_master_route_id
        },
        "tags": {
            "name": name,
            "ref": ref,
            "network": 'STL',
            "operator": 'STL',
            "type": "route_master",
            "public_transport:version": 2
        },
        "members": members
    }
    route_master_relations.append(route_master_relation)

In [ ]:
route_master_relations[0]

In [ ]:
filtered_routes['402']

In [ ]:
for ref, routes in filtered_routes.items():
    for route in routes:
        members = []
        
        stops = route['stops']        
        stops.sort(key=itemgetter('sequence'))
        
        for stop in stops:
            member = {
                "props": {
                    "ref": stop['osm_id'],
                    "type": "node",
                    "role": "platform"
                }
            }
            members.append(member)
            
        route_relation = {
            "props": {
                "id": route['props']['id']
            },
            "tags": route['tags'],
            "members": members
        }
        route_relations.append(route_relation)

In [ ]:
route_relations[0]

### Write JOSM files

In [ ]:
output_file = os.path.join(output_dir, 'gtfs_laval.xml')

if os.path.exists(output_file):
    os.remove(output_file)

root = ET.Element("osm")
root.set('version', '0.6')

min_lon, max_lon, min_lat, max_lat = multi_point.GetEnvelope()

bounds = ET.SubElement(root, 'bounds')
bounds.set('minlat', str(min_lat))
bounds.set('minlon', str(min_lon))
bounds.set('maxlat', str(max_lat))
bounds.set('maxlon', str(max_lon))

for stop in unique_stops:
    node = ET.SubElement(root, 'node')
    node.set('version', '1')
    
    for key, value in stop.items():
        if key == 'tags':
            for k, v in stop['tags'].items():
                tag = ET.SubElement(node, 'tag')
                tag.set('k', str(k))
                tag.set('v', str(v))
        if key == 'props':
            for k, v in stop['props'].items():
                node.set(k, v)
                
for route_relation in route_relations:
    relation = ET.SubElement(root, 'relation')
    relation.set('version', '1')
    
    for key, value in route_relation.items():
        if key == 'tags':
            for k, v in route_relation['tags'].items():
                tag = ET.SubElement(relation, 'tag')
                tag.set('k', str(k))
                tag.set('v', str(v))
        if key == 'props':
            for k, v in route_relation['props'].items():
                relation.set(k, str(v))
        if key == 'members':
            for member in route_relation['members']:
                mem = ET.SubElement(relation, 'member')
                
                for key, value in member.items():
                    if key == 'props':
                        for k, v in member['props'].items():
                            mem.set(k, v)
                            
for route_master_relation in route_master_relations:
    relation = ET.SubElement(root, 'relation')
    relation.set('version', '1')
    
    for key, value in route_master_relation.items():
        if key == 'tags':
            for k, v in route_master_relation['tags'].items():
                tag = ET.SubElement(relation, 'tag')
                tag.set('k', str(k))
                tag.set('v', str(v))
        if key == 'props':
            for k, v in route_master_relation['props'].items():
                relation.set(k, str(v))
        if key == 'members':
            for member in route_master_relation['members']:
                mem = ET.SubElement(relation, 'member')
                
                for key, value in member.items():
                    if key == 'props':
                        for k, v in member['props'].items():
                            mem.set(k, str(v))
                            
tree = ET.ElementTree(root)
tree.write(output_file, encoding='unicode')

In [ ]:
for route_master_relation in route_master_relations:
    print(route_master_relation['tags'])